In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/11.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/12.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112400567,ow5aQaW,0
1,1112306051,lDLm5EG,0
2,1111609950,xkd0Ebz,0
3,1112289649,zv43m38,0
4,1112297123,1YqKoR,0
5,1112348520,Nz0K25B,0
6,1112289881,ZDrda4E,0
7,1111954700,Z93Pdb,0
8,1112423684,VNPJGEm,0
9,1112295743,eJdZz9,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502050


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
13041706,1112398139,0zPvZ6E,0,NO_DECLARA,200.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
4238973,1111857718,akjGG6E,0,MASC,28.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,NaN
11454783,1112507735,wVjjPkl,0,MASC,43.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
5807175,1112315601,VNkj69l,1,MASC,23.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
4375457,1112352091,a4GM4J,1,MASC,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10412576,1112315202,qe2J83x,1,MASC,24.0,-,En Curso,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2712995,1112373598,3NP46pw,0,FEM,29.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
11313223,1112254576,ZDlL0GP,0,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,NaN
12143834,1111935485,MVr193x,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,NaN
15308004,1112493896,JBrae6r,0,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,True,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
12154349,1111599916,2zPr6NQ,0,FEM,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
6236546,1112378413,bOzOa1B,0,FEM,22.0,-,-,-,-,-,...,False,True,False,False,False,True,False,False,False,0.000000
9665944,1112355344,owaX1mL,1,MASC,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
8554809,1112272737,6r5LR9L,1,FEM,34.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
2671821,1112283239,Yj5kj2Z,1,MASC,26.0,-,-,-,-,-,...,False,False,False,True,False,False,False,False,False,0.000000
1579152,1112308336,EzlXW9J,1,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
15112848,1112364678,rmePlBb,0,FEM,27.0,-,-,-,En Curso,Graduado,...,False,True,False,False,False,False,False,True,False,0.000000
697739,1112393839,PmB8DP0,1,MASC,34.0,-,-,Graduado,En Curso,Graduado,...,False,False,False,False,False,False,False,False,False,0.000782
10196547,1112384723,ZO5z6P,0,FEM,49.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.000000
584936,1112369306,jk6RE6b,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281915 Test:  804312


In [16]:
gc.collect()

161

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=24000,max_depth=17,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.689875
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.685727
[3]	valid_0's binary_logloss: 0.681428
[4]	valid_0's binary_logloss: 0.678099
[5]	valid_0's binary_logloss: 0.673923
[6]	valid_0's binary_logloss: 0.670364
[7]	valid_0's binary_logloss: 0.666424
[8]	valid_0's binary_logloss: 0.662865
[9]	valid_0's binary_logloss: 0.661205
[10]	valid_0's binary_logloss: 0.658288
[11]	valid_0's binary_logloss: 0.654709
[12]	valid_0's binary_logloss: 0.651086
[13]	valid_0's binary_logloss: 0.648507
[14]	valid_0's binary_logloss: 0.646922
[15]	valid_0's binary_logloss: 0.644197
[16]	valid_0's binary_logloss: 0.64095
[17]	valid_0's binary_logloss: 0.637465
[18]	valid_0's binary_logloss: 0.634906
[19]	valid_0's binary_logloss: 0.631838
[20]	valid_0's binary_logloss: 0.628455
[21]	valid_0's binary_logloss: 0.625142
[22]	valid_0's binary_logloss: 0.622941
[23]	valid_0's binary_logloss: 0.620483
[24]	valid_0's binary_logloss

In [18]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9159075938190114


In [19]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV22.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV22.pkl']

In [20]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
87204,40034,1112418466,5LBmEq,MASC,42.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
83676,32362,1112393398,111K4O,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
37176,57067,1112451030,zv64e85,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.019231
83955,32667,1112396463,890zDj,FEM,42.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,0.000000
74165,20253,1112252531,6rdWjL,MASC,35.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
19024,77131,1112464306,ZDr22ZP,MASC,24.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.017241
36795,23552,1112306545,NGA26B,FEM,44.0,-,-,-,-,-,...,False,True,True,False,False,False,False,True,False,0.000000
3423,61939,1112455845,1QP1dK3,MASC,21.0,-,-,-,-,En Curso,...,False,True,False,False,False,False,False,True,False,0.000000
34016,1315,1110539976,wVYN9RZ,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
10432,87188,1112466797,YRPbzP,MASC,36.0,-,-,-,-,Graduado,...,False,True,True,True,False,False,False,False,False,0.005068


In [21]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.93542953, 0.06457047],
       [0.90400185, 0.09599815],
       [0.88774522, 0.11225478],
       ...,
       [0.99403152, 0.00596848],
       [0.99236907, 0.00763093],
       [0.99803407, 0.00196593]])

In [22]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [23]:
prediccion['sepostulo']=y_final2

In [24]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.064570
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.095998
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.112255
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.024666
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.038032


In [25]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.064570
1,1,0.095998
2,2,0.112255
3,3,0.024666
4,4,0.038032


In [26]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion47.csv",index=False)

In [27]:
prediccion.sepostulo.mean()

0.5221680971093267